# Description
After running K fold to select those data which are predicted into a wrong category, we implemented mannual recorrection. This script is to apply those correction to the original files. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
# model_list = ['CutDebt','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
df_others = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set.csv')
df_other_matrix = pd.read_csv('../../MLModel/data/others/strategy_mat.csv')
num2text = df_other_matrix.set_index('label').category.drop_duplicates().to_dict()

df_all_original = {}
for each in model_list:
    df_all_original[each] = pd.read_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(each))

In [3]:
for model in model_list:
    df_correction = pd.read_csv('../../MLModel/data/{}/labelNeedsCorrections0719.csv'.format(model))
    other_label = max(df_all_original[model].label) + 1
    non_other_label = list(range(other_label))
    df_all_original[model] = df_all_original[model][df_all_original[model].label != other_label]
    print('{} -- {}'.format(model,len(df_all_original[model])))
    print('length of other: {}'.format(len(df_others)))
    append_other_text = []
    append_other_category = []

    append_function_text = {}
    append_function_category = {}

    original_remove_index = {}
    others_remove_index = []

    count_case1 = 0
    count_case2 = 0
    count_case3 = 0
    count_case4 = 0
    count_case5 = 0
    count_case6 = 0
    count_case7 = 0
    for index in range(len(df_correction)):
        ############ get data #####################
        cur_data = df_correction.iloc[index]
        data_from = cur_data['from']
        label = cur_data['label']
        original_text = cur_data['original_text']
        new_label = cur_data['new_label']
        original_label = cur_data['original_label']
        ########### apply to original files #######
        if model in ['CutDebt','Installment'] and new_label == 110:
            count_case4 += 1
            if label in non_other_label:
                ### insert into others
                append_other_text.append(original_text)
                append_other_category.append(num2text[new_label])
                ### remove from original
                rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
                if original_remove_index.get(data_from) is None:
                    original_remove_index[data_from] = []
                original_remove_index[data_from].extend(list(rindex))
            else:
                ### update others
                rindex = df_others[df_others['文本']==original_text].index.values
                if len(rindex) == 0:
                    ### insert into others
                    append_other_text.append(original_text)
                    append_other_category.append(num2text[new_label])
                else:    
                    df_others.loc[rindex,'类别'] = num2text[110]
            continue
            
            
            
        # move from functional to other
        if label in non_other_label and new_label >= 100:
            count_case1 += 1
            ### insert into others
            append_other_text.append(original_text)
            append_other_category.append(num2text[new_label])
            # count 109
            if new_label == 109:
                count_case7 += 1

            ### remove from original
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))
            
            
            
        # delete non_other_label
        elif label in non_other_label and new_label == -1:
            ## remove from original
            count_case5 += 1
            rindex = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            if original_remove_index.get(data_from) is None:
                original_remove_index[data_from] = []
            original_remove_index[data_from].extend(list(rindex))
            
        # delete other_label
        elif label == other_label and new_label == -1:
            if original_label != 109:
                count_case6 += 1
                rindex_other = df_others[(df_others['类别']==num2text[int(original_label)]) &(df_others['文本']==original_text)].index.values
                others_remove_index.extend(list(rindex_other))
        


        # update from functional 1 to functional 2
        elif label in non_other_label and new_label in non_other_label and label!= new_label:
            count_case2 += 1
            # update original data
            indexes = df_all_original[data_from][(df_all_original[data_from].label==label) &(df_all_original[data_from].split_text==original_text)].index.values
            df_all_original[data_from].loc[indexes,'label'] = new_label
            
            
            
            
        # update from other to functional
        elif label == other_label and new_label in non_other_label:
            count_case3+=1
            if append_function_text.get(model) is None:
                append_function_text[model]=[]
                append_function_category[model]=[]
            append_function_text[model].append(original_text)
            append_function_category[model].append(new_label)
            
            # remove from others
            if original_label != 109:
                rindex_other = df_others[(df_others['类别']==num2text[int(original_label)]) &(df_others['文本']==original_text)].index.values
                others_remove_index.extend(list(rindex_other))
            

    others_append = pd.DataFrame({'文本':append_other_text,'类别':append_other_category})
    function_append = pd.DataFrame({'split_text':append_function_text,'label':append_function_category})
    
    
    
    

    #### apply to self and others
    #### drop logic
    for each_key in original_remove_index:
        indexes = original_remove_index[each_key]
        df_all_original[each_key] = df_all_original[each_key].drop(index=indexes)
        print('{} -- {}'.format(each_key,len(df_all_original[each_key])))
    ####end drop
    #### drop others
    
    df_others = df_others.drop(index=others_remove_index)
    ### end drop others
    df_others = pd.concat([df_others,others_append],sort=True).reset_index(drop=True)
    print('length of other: {}'.format(len(df_others)))
    for each_key in append_function_text:
        function_append = pd.DataFrame({'split_text':append_function_text[each_key],
                                        'label':append_function_category[each_key]})

        df_all_original[each_key] = pd.concat([df_all_original[each_key],function_append],sort=True).reset_index(drop=True)
        print('{} -- {}'.format(each_key,len(df_all_original[each_key])))

    print('case 1 <function to others> has data : {}'.format(count_case1))
    print('case 2 <function to function> has data : {}'.format(count_case2))
    print('case 3 <others to function> has data : {}'.format(count_case3))
    print('case 4 has data : {}'.format(count_case4))
    print('case 5 <deleted from function> delted data: {}'.format(count_case5))
    print('case 6 <deleted from other>: {}'.format(count_case6))
    print('case 7 <function to others109>: {}'.format(count_case7))
    print('total corrected is: {}'.format(count_case1+count_case2+count_case3+count_case4))
    print('total file length is: {}'.format(len(df_correction)))
    print('----------------------------------------------------------------')


CutDebt -- 4204
length of other: 5440
CutDebt -- 3915
Installment -- 4258
length of other: 5452
CutDebt -- 4253
case 1 <function to others> has data : 283
case 2 <function to function> has data : 27
case 3 <others to function> has data : 338
case 4 has data : 14
case 5 <deleted from function> delted data: 21
case 6 <deleted from other>: 0
case 7 <function to others109>: 29
total corrected is: 662
total file length is: 1680
----------------------------------------------------------------
IDClassifier -- 2416
length of other: 5452
IDClassifier -- 2228
length of other: 5554
IDClassifier -- 2326
case 1 <function to others> has data : 121
case 2 <function to function> has data : 14
case 3 <others to function> has data : 98
case 4 has data : 0
case 5 <deleted from function> delted data: 19
case 6 <deleted from other>: 5
case 7 <function to others109>: 10
total corrected is: 233
total file length is: 748
----------------------------------------------------------------
IfKnowDebtor -- 2626
len

In [38]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
total = 0
for model in model_list:
    print(model)
    print(df_all_original[model].drop_duplicates().label.value_counts())
    total += np.sum(df_all_original[model].drop_duplicates().label.value_counts())
    print('===================')
print(total)

CutDebt
1.0    1702
0.0     973
Name: label, dtype: int64
IDClassifier
1    773
0    589
Name: label, dtype: int64
IfKnowDebtor
0    981
1    734
Name: label, dtype: int64
Installment
1.0    1591
0.0    1191
Name: label, dtype: int64
ConfirmLoan
0    814
1    567
Name: label, dtype: int64
WillingToPay
1.0    1870
0.0    1102
2.0     536
Name: label, dtype: int64
13423


In [37]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
total = 0
for model in model_list:
    print(model)
    print(df_all_original[model].label.value_counts())
    total += np.sum(df_all_original[model].label.value_counts())
    print('===================')
print(total)

CutDebt
1.0    2428
0.0    1811
Name: label, dtype: int64
IDClassifier
1    1318
0    1008
Name: label, dtype: int64
IfKnowDebtor
1    1366
0    1353
Name: label, dtype: int64
Installment
1.0    2375
0.0    1915
Name: label, dtype: int64
ConfirmLoan
0    876
1    623
Name: label, dtype: int64
WillingToPay
1.0    2686
0.0    1769
2.0     851
Name: label, dtype: int64
20379


In [6]:
len(df_others)

6246

In [8]:
df_others = df_others.drop_duplicates()

In [9]:
len(df_others.drop_duplicates())

5606

In [10]:
# Save 
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
for model in model_list:
    df = df_all_original[model].copy()
    df['label'] = df['label'].astype('int')
    df.to_csv('../../MLModel/data/{}/mock_up_data1.csv'.format(model),index=False,encoding='utf8')
    print(model)
df_others.to_csv('../../MLModel/data/others/irrelevant_response_training_set.csv',index=False,encoding='utf8')

CutDebt
IDClassifier
IfKnowDebtor
Installment
ConfirmLoan
WillingToPay


# Other Data 109 . Diff 0723 - 0729

In [12]:
other_109_0729 = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set_109_0729.csv')
other_109_0723 = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set_109_0723.csv')

In [18]:
diff_text = []
diff_category = []
for index in range(len(other_109_0729)):
    data = other_109_0729.iloc[index]
    text = data['文本']
    category = data['类别']
    finds = other_109_0723[(other_109_0723['文本']==text) &(other_109_0723['类别']==category)]
    if len(finds) == 0:
        diff_text.append(text)
        diff_category.append(category)
diff = pd.DataFrame({'文本':diff_text,'类别':diff_category})
print(diff.shape)
    

(128, 2)


In [20]:
diff.to_csv('../../MLModel/data/others/diff_training_set_109_0723vs0729.csv',index=False,encoding='utf8')

# Experiment

In [26]:
tanshu_ID = pd.read_csv('../../MLModel/data/IDClassifier/labelNeedsCorrections0719.csv')

In [28]:
tanshu_ID

,from,label,original_label,original_text,new_label,Unnamed: 5
0,说出目的,2,101.0,我不是，你有啥事情？我帮你找他,2,1.0
1,请求等下打来,2,104.0,你好，我在忙,2,NaN
2,故意岔开话题,2,109.0,啊？,103,NaN
3,IDClassifier,1,NaN,不认识,-1,NaN
4,故意岔开话题,2,109.0,我是喜欢你,2,NaN
5,IDClassifier,1,NaN,我听不懂,1,NaN
6,IDClassifier,0,NaN,我有事你赶紧说,101,1.0
7,说出目的,2,101.0,嗯，咋了,0,NaN
8,IDClassifier,1,NaN,错号，不认识。,1,NaN
9,IDClassifier,0,NaN,我正在开车，一会联系,104,NaN


In [30]:
dif = tanshu_ID[tanshu_ID.label !=tanshu_ID.new_label]

In [31]:
dif.to_csv('../../MLModel/data/IDClassifier/reports.csv',index=False,encoding='utf8')

# Meeting 0729 
need to clean up all functional data

In [45]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
read = '../../MLModel/data/{}/mock_up_data1.csv'
save = '../../MLModel/data/{}/mock_up_data_clean_0730.csv'
for each in model_list:
    df = pd.read_csv(read.format(each),encoding='utf8')
    print(each)
    print('before drop: {}'.format(len(df)))
    df = df.drop_duplicates()
    df = df.sample(frac=1,random_state=19)
    print('after drop: {}'.format(len(df)))
    df.label = df.label.astype('int')
    df['classifier'] = each
    df['new_label'] = 0
    df.to_csv(save.format(each),encoding='utf8',index=False)
    

CutDebt
before drop: 4239
after drop: 2675
IDClassifier
before drop: 2327
after drop: 1363
IfKnowDebtor
before drop: 2719
after drop: 1715
Installment
before drop: 4290
after drop: 2782
ConfirmLoan
before drop: 1499
after drop: 1381
WillingToPay
before drop: 5304
after drop: 3506


In [40]:
df['new_label']

,label,split_text,classifier
0,0,我等会就处理,WillingToPay
1,0,已经还了，提示正在交易中,WillingToPay
2,0,要得,WillingToPay
3,0,好的，好的，好的,WillingToPay
4,0,行，你放一万个心,WillingToPay
5,0,行你放心,WillingToPay
6,0,行，行，行,WillingToPay
7,0,嗯嗯，好的,WillingToPay
8,1,我现在没钱,WillingToPay
9,2,这太多了能减免一点吗,WillingToPay
